# Строим простую картограмму Pandas+Vincent

https://habr.com/post/198974/

In [1]:
!pip install -q vincent bs4

[**Vincent**](https://vincent.readthedocs.io/en/latest/index.html) — это модуль предназначенный для трасляции данных из python в JavaScript библиотеки для визуализации [D3js](http://d3js.org/) и [Vega](http://trifacta.github.io/vega/), которые в свою очередь дают большие возможности для интерактивной визуализации данных.
Т.е. таким образом мы можем производить анализ на python, а графику к результатам мы сможем строить на js. Например это может быть удобно для визуализации каких-либо георафических данных и нанесения их на карту. Кроме того vincent обладает интеграцией с `IPython Notebook`, и, так же как и `matplotlib`, может выводить графику непосредственно в ней.
В качестве демонстрации возможностей данного модуля, я предлагаю реализовать 2 задачи:

Покажем динамику среднедушевого дохода населения и Центральному и Приволжскому федерельным округам

Покажем на карте РФ распределение по субъектам РФ среднедушевого дохода за 2010

В качестве исходных данных возьмем статисткику [сайта](http://www.gks.ru/bgd/regl/b12_14p/IssWWW.exe/Stg/d01/05-02.htm) Росстата.

In [2]:
import pandas as pd
import vincent

In [3]:
#!wget "http://www.gks.ru/bgd/regl/b12_14p/IssWWW.exe/Stg/d01/05-02.htm" -O AVGPeopleProfit.htm
!ls *.htm

AVGPeopleProfit.htm


In [4]:
! (iconv -f cp1251 -t utf8 -c < AVGPeopleProfit.htm | tail -n +10 | head) 2>/dev/null

Для загрузки данных используем функцию
[read_html()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.html.read_html.html?highlight=read_html#pandas.io.html.read_html).
В качестве параметров в нашем случае передается 3 аргумента:
- Адрес html страницы
- Номер строки, содержащей имена столбцов
- Номер столбца, который будет использоваться в виде индекса

In [5]:
stat = pd.read_html('AVGPeopleProfit.htm', header=0, index_col=0)[0]
stat

,1990,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,"Место, занимае-мое в Российской Федерации 2011"
Российская Федерация,0217,2281,3062,3947,5167,6399,8088,10155,12540,14864,16895,18951.0,20755.0,NaN
Центральный федеральный округ,NaN,3231,4300,5436,7189,8900,10902,13570,16631,18590,21931,24645.0,27091.0,1.0
Белгородская область,0195,1555,2121,2762,3357,4069,5276,7083,9399,12749,14147,16993.0,18800.0,24.0
Брянская область,0202,1312,1818,2452,3136,3725,4788,6171,7626,10083,11484,13358.0,15348.0,52.0
Владимирская область,0190,1280,1666,2158,2837,3363,4107,5627,7015,9480,10827,12956.0,14312.0,64.0
Воронежская область,0186,1486,2040,2597,3381,4104,5398,6862,8307,10587,11999,13883.0,15871.0,48.0
Ивановская область,0193,1038,1298,1778,2292,2855,3480,4457,5684,8343,9351,11124.0,13006.0,74.0
Калужская область,0202,1311,1782,2503,3335,4163,5343,6925,9185,11612,13380,15477.0,17557.0,31.0
Костромская область,0193,1439,1905,2516,3094,3837,4985,6398,7857,9608,10696,13315.0,14823.0,57.0
Курская область,0187,1465,1980,2699,3373,4241,5218,6751,8687,11524,12801,14685.0,16387.0,41.0


Как можно заметить нужна будет небольшая обработка таблицы, т.к. в ней присутствует один столбец без названия и один столбец с пустыми значениями. Выберем нужные нам столцы (со 2 по 13) остальные стобцы и поместим их в новый DataFrame.

In [6]:
stat = stat[stat.columns[1:13]]
stat

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
Российская Федерация,2281,3062,3947,5167,6399,8088,10155,12540,14864,16895,18951.0,20755.0
Центральный федеральный округ,3231,4300,5436,7189,8900,10902,13570,16631,18590,21931,24645.0,27091.0
Белгородская область,1555,2121,2762,3357,4069,5276,7083,9399,12749,14147,16993.0,18800.0
Брянская область,1312,1818,2452,3136,3725,4788,6171,7626,10083,11484,13358.0,15348.0
Владимирская область,1280,1666,2158,2837,3363,4107,5627,7015,9480,10827,12956.0,14312.0
Воронежская область,1486,2040,2597,3381,4104,5398,6862,8307,10587,11999,13883.0,15871.0
Ивановская область,1038,1298,1778,2292,2855,3480,4457,5684,8343,9351,11124.0,13006.0
Калужская область,1311,1782,2503,3335,4163,5343,6925,9185,11612,13380,15477.0,17557.0
Костромская область,1439,1905,2516,3094,3837,4985,6398,7857,9608,10696,13315.0,14823.0
Курская область,1465,1980,2699,3373,4241,5218,6751,8687,11524,12801,14685.0,16387.0


Итак, давайте приступим к выполнениию первой задачи по визуализации данных 2 округов. Чтобы получить данные, на основе которых мы построим график, необходимо выбрать интересующие нас округа (Московский и Приволжский), а затем транспонировать полученную таблицу. Сделать это можно так:

In [7]:
fo = ['Приволжский федеральный округ', 'Центральный федеральный округ']
fostat = stat[stat.index.isin(fo)].transpose()

В приведенном выше коде, мы сначала фильтруем наш набор данных по нужным нам округам при момощи функции [isin()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.isin.html#pandas.Series.isin), которая проверяет значение столбца в заданее заданном списке (аналог оператора IN в SQL). Затем используем функцию [transpose()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.transpose.html?highlight=transpose#pandas.DataFrame.transpose) для транспонирования полученного набор данных и записываем результат в новый DataFrame.

In [8]:
fostat

,Центральный федеральный округ,Приволжский федеральный округ
2000,3231,1726
2001,4300,2319
2002,5436,3035
2003,7189,3917
2004,8900,4787
2005,10902,6229
2006,13570,8014
2007,16631,9959
2008,18590,12392
2009,21931,13962


Как можно заметить названия индексы в таблице у нас теперь равны номеру года в числовом формате. Это не очень удобно, поэтому давайте поменяем индекс на формат даты.

Фукция [set_index()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.set_index.html?highlight=set_index#pandas.DataFrame.set_index)
используется для задания нового индекса в DataFrame.
В нашем случае ей передается 2 параметра:
- Список новых значений индекса(может быть также названием столбца)
- Параметр означает, что мы заменяем индекс в текущем наборе, если он будет False индекс не сохранится

In [9]:
fostat.set_index(pd.date_range('2000','2011', freq='AS'), inplace=True)
fostat

,Центральный федеральный округ,Приволжский федеральный округ
2000-01-01,3231,1726
2001-01-01,4300,2319
2002-01-01,5436,3035
2003-01-01,7189,3917
2004-01-01,8900,4787
2005-01-01,10902,6229
2006-01-01,13570,8014
2007-01-01,16631,9959
2008-01-01,18590,12392
2009-01-01,21931,13962


Теперь данные у нас полностью готовы для построения графика. Итак если вы работаете в IPython Notebook и хотите видеть результат в реальном времени то для интеграции необходимо вызвать функцию `initialize_notebook()`. Выглядеть это будет следующим образом:

In [10]:
vincent.core.initialize_notebook()

Теперь нам нужно создать объект соответствующий типу диаграммы
(полный список объектов можно увидеть в [документации](https://vincent.readthedocs.org/en/latest/)).
В нашем случае это будут линейный график.

In [11]:
line = vincent.Line(fostat)               # создаем объект графика
line.axis_titles(x='Год', y='тыс. руб')   # задаем названия осей
line.legend(title='ЦФО vs ПФО')          # выводим легенду и задаем ей заголовок

Вывести график можно с помощью функции `display()`:

In [12]:
#line.display()

### Построение картограммы

Ну что же с первой задачей мы справились. Теперь давайте перейдем ко второй.
Для ее решения нам понадобится `TopoJSON` файл с картой РФ, а также справочник регионов.
Подробно о том как их получить и что это такое можно почитать [здесь](https://habr.com/post/181766/).

In [13]:
#! wget -O russia-region-names.tsv https://raw.githubusercontent.com/KoGor/Maps.GeoInfo/master/russia-region-names.tsv -nv
#! wget -O russia.json https://raw.githubusercontent.com/KoGor/Maps.GeoInfo/master/russia_1e-7sr.json -nv
! ls russia*

russia.json  russia-region-names.tsv


In [14]:
! head russia-region-names.tsv

Белгородская область	RU-BEL
Брянская область	RU-BRY
Владимирская область	RU-VLA
Воронежская область	RU-VOR
Ивановская область	RU-IVA
Калужская область	RU-KLU
Костромская область	RU-KOS
Курская область	RU-KRS
Липецкая область	RU-LIP
Город Москва	RU-MOW


In [15]:
! ( cat russia.json | jq . | head ) 2> /dev/null

{
  "type": "Topology",
  "transform": {
    "scale": [
      0.03600360003702599,
      0.0040674580654071245
    ],
    "translate": [
      -179.9999967702229,
      41.18684289776669


Для начала давайте загрузим справочник регионов с помощью `read_csv`,
описанной в одной из [предыдущих](https://habr.com/post/196980/) статей.

Тут появилось несколько дополнительных параметров:
- `index_col` — задает номер столбца который будет использоватся в качестве индекса
- `header` — в нашем случае означает, что для определения заголовков мы не используем строки из файла
- `names` — получает список, элементы которого будут названиями столбцов
- `encoding` — задает кодировку в котрой хранится файл

In [16]:
spr = pd.read_csv('russia-region-names.tsv',
                  '\t',
                  index_col=0, header=None,
                  names = ['name','code'],
                  encoding='utf-8')
spr

,code
name,
Белгородская область,RU-BEL
Брянская область,RU-BRY
Владимирская область,RU-VLA
Воронежская область,RU-VOR
Ивановская область,RU-IVA
Калужская область,RU-KLU
Костромская область,RU-KOS
Курская область,RU-KRS
Липецкая область,RU-LIP


Если мы посмотрим внимательно, на наш набор данных `stat`, то можно увидеть, что его некоторые элементы содержат сноски тип `'1)'` и `'2)'`, которые при парсинге с помощью `read_html()` перекодировались в обычные символы и добавились в конце соответствующих строк в индексом сотлбце.

In [17]:
stat = pd.read_html('AVGPeopleProfit.htm', header=0, index_col=0)[0]
stat = stat[stat.columns[1:13]]
stat

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
Российская Федерация,2281,3062,3947,5167,6399,8088,10155,12540,14864,16895,18951.0,20755.0
Центральный федеральный округ,3231,4300,5436,7189,8900,10902,13570,16631,18590,21931,24645.0,27091.0
Белгородская область,1555,2121,2762,3357,4069,5276,7083,9399,12749,14147,16993.0,18800.0
Брянская область,1312,1818,2452,3136,3725,4788,6171,7626,10083,11484,13358.0,15348.0
Владимирская область,1280,1666,2158,2837,3363,4107,5627,7015,9480,10827,12956.0,14312.0
Воронежская область,1486,2040,2597,3381,4104,5398,6862,8307,10587,11999,13883.0,15871.0
Ивановская область,1038,1298,1778,2292,2855,3480,4457,5684,8343,9351,11124.0,13006.0
Калужская область,1311,1782,2503,3335,4163,5343,6925,9185,11612,13380,15477.0,17557.0
Костромская область,1439,1905,2516,3094,3837,4985,6398,7857,9608,10696,13315.0,14823.0
Курская область,1465,1980,2699,3373,4241,5218,6751,8687,11524,12801,14685.0,16387.0


Кроме того перед именами городов в нашем наборе строит буква `'г. '`, а в справочнике ее нет.

Все эти мелочи влияют на то, что когда мы будем объединять набор со стат. данными и справочник, чтобы подтянуть коды к регионам, у нас будут регионы без кода.
Исправить это можно следующим образом:

In [18]:
new_index = stat.index.to_series()
new_index

Российская Федерация                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Российская Федерация
Центральный федеральный округ                                                                                                                                                                                                      

In [19]:
new_index = stat.index.to_series()
new_index = new_index.str.replace('[123][)]','')
new_index = new_index.str.replace('^г\. ','Город ')
new_index

Российская Федерация                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Российская Федерация
Центральный федеральный округ                                                                                                                                                                                                      

Теперь нам надо значения индекса заменить значениями из нового набора.

In [20]:
stat.set_index(new_index, inplace=True)
stat

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
Российская Федерация,2281,3062,3947,5167,6399,8088,10155,12540,14864,16895,18951.0,20755.0
Центральный федеральный округ,3231,4300,5436,7189,8900,10902,13570,16631,18590,21931,24645.0,27091.0
Белгородская область,1555,2121,2762,3357,4069,5276,7083,9399,12749,14147,16993.0,18800.0
Брянская область,1312,1818,2452,3136,3725,4788,6171,7626,10083,11484,13358.0,15348.0
Владимирская область,1280,1666,2158,2837,3363,4107,5627,7015,9480,10827,12956.0,14312.0
Воронежская область,1486,2040,2597,3381,4104,5398,6862,8307,10587,11999,13883.0,15871.0
Ивановская область,1038,1298,1778,2292,2855,3480,4457,5684,8343,9351,11124.0,13006.0
Калужская область,1311,1782,2503,3335,4163,5343,6925,9185,11612,13380,15477.0,17557.0
Костромская область,1439,1905,2516,3094,3837,4985,6398,7857,9608,10696,13315.0,14823.0
Курская область,1465,1980,2699,3373,4241,5218,6751,8687,11524,12801,14685.0,16387.0


Теперь мы можем объединить наш набор данных со справочником, чтобы получить коды регионов:

In [21]:
region_profit = stat.join(spr, how='inner')
region_profit

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,code
Белгородская область,1555,2121,2762,3357,4069,5276,7083,9399,12749,14147,16993.0,18800.0,RU-BEL
Брянская область,1312,1818,2452,3136,3725,4788,6171,7626,10083,11484,13358.0,15348.0,RU-BRY
Владимирская область,1280,1666,2158,2837,3363,4107,5627,7015,9480,10827,12956.0,14312.0,RU-VLA
Воронежская область,1486,2040,2597,3381,4104,5398,6862,8307,10587,11999,13883.0,15871.0,RU-VOR
Ивановская область,1038,1298,1778,2292,2855,3480,4457,5684,8343,9351,11124.0,13006.0,RU-IVA
Калужская область,1311,1782,2503,3335,4163,5343,6925,9185,11612,13380,15477.0,17557.0,RU-KLU
Костромская область,1439,1905,2516,3094,3837,4985,6398,7857,9608,10696,13315.0,14823.0,RU-KOS
Курская область,1465,1980,2699,3373,4241,5218,6751,8687,11524,12801,14685.0,16387.0,RU-KRS
Липецкая область,1772,2261,2776,3557,4406,5591,7611,9472,12085,14487,15936.0,16811.0,RU-LIP
Московская область,1824,2626,3546,4409,5753,7445,10515,14034,19047,20064,22641.0,25605.0,RU-MOS


Перейдем к непосредственному построению карты и нанесению на нее данных.
Для начача нам нужно создать словарь с описание нашей карты:

In [22]:
geo_data = [{'name': 'rus',           # имя карты
             'url': './russia.json',  # путь до TopoJSON файла с картой
             'feature': 'russia'}]    # имя объекта из файла карты

Теперь давайте создадим объект нашей карты и привяжем к нему наши данные.
Сделать это можно функцией`Map()`:

In [23]:
vis = vincent.Map(data = region_profit, geo_data = geo_data,
                  scale = 700, projection = 'conicEqualArea', rotate = [-105,0], center = [-10, 65],
                  data_bind = '2011',
                  data_key = 'code',
                  map_key = {'rus': 'properties.region'}
                 )

AttributeError: 'tuple' object has no attribute '__name__'

In [24]:
# ????????